# Filled Area Plot

## Basic Filled Area Plot

# Multiple lines with filled area between them

This interactive plot displays the minimum and maximum temperatures throughout a week. The blue line represents the daily minimum temperatures, while the orange line shows the maximum temperatures. The shaded area between these lines illustrates the daily temperature range.
- Days with the highest and lowest temperatures
- Days with the largest temperature fluctuations
- Overall temperature trends throughout the week


## Filled Area Plot with Customization Calulation

This section demonstrates how to create a filled area plot using Plotly Express with additional customization such as adding a moving average line and shaded volatility bands. The example uses stock price data.

- The filled area plot shows the stock price over time, with the area between the moving average and the stock price shaded to indicate volatility.
- The moving average line smooths out the stock price fluctuations, making it easier to identify trends.
- The shaded area between the moving average and the stock price represents the volatility of the stock, with wider bands indicating higher volatility.
- The example also includes hover information to display the stock price and moving average values when hovering over the plot.

## Stacked Area Plot
